In [414]:
import pandas as pd
import numpy as np
import datetime
from sksurv.preprocessing import OneHotEncoder
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV

In [212]:
def fix_str_float(ds, col):
    ds[col] = ds[col].str.replace(r'[^0-9\.]','')
    ds[col] = np.where(ds[col]=='',np.nan,ds[col])
    ds[col] = ds[col].astype('float32')
    return ds[col].astype('float32')

In [248]:
# clicks
clicks_df = pd.read_csv('data/clicks.csv', low_memory=False,dtype={'advertiser_id':'int32','action_id':'float32','source_id':'int32','country_code':'category',\
                                                 'latitude':'float32','longitude':'float32','carrier_id':'float32','os_minor':'category',\
                                                  'os_major':'category','specs_brand':'category','timeToClick':'float32','ref_type':'category'\
                                                                  ,'ref_hash':'object'})

clicks_df['touchX'] = fix_str_float(clicks_df,'touchX')
clicks_df['touchY'] = fix_str_float(clicks_df,'touchY')
clicks_df['created'] = pd.to_datetime(clicks_df['created'])
#events
events_df = pd.read_csv('data/events.csv', low_memory=False, dtype={'event_id':'int32','ref_type':'category','application_id':'category',\
                                                                                            'attributed':'bool','device_countrycode':'category','device_city':'category',\
                                                                                            'trans_id':'category','carrier':'category','device_os':'category',\
                                                                                            'connection_type':'category'})
events_df['date'] = pd.to_datetime(events_df['date'])
events_df['wifi'].astype('bool', inplace=True)
events_df.drop(columns=['device_countrycode','session_user_agent','ip_address','device_language'], inplace=True)
# installs
installs_df = pd.read_csv('data/installs.csv', low_memory=False, dtype={'ref_type':'category','application_id':'category',\
                                                      'device_brand':'category','ref_hash':'object','wifi':'category'})
installs_df['kind'] = installs_df['kind'].str.lower()
installs_df['kind'] = installs_df['kind'].astype('category')
installs_df.drop(columns=['session_user_agent','ip_address','device_language','device_model'], inplace=True)
installs_df['created'] = pd.to_datetime(installs_df['created'])
installs_df.drop(['device_countrycode'], axis=1, inplace=True)
# auctions
auctions_df = pd.read_csv('data/auctions.csv', low_memory=False, dtype={'country':'category','platform':'category',\
                                                                        'ref_type_id':'category','source_id':'category','device_id':'object'})

auctions_df['date'] = pd.to_datetime(auctions_df['date'])
print('setup done')

setup done


In [4]:
installs_df.head(5)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_brand,device_model,user_agent,event_uuid,kind,wifi,trans_id
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,NaN,False,True,NaN,3.739127e+17,NaN,79837499-2f2a-4605-a663-e322f759424f,app_open,NaN,NaN
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,NaN,False,False,NaN,7.805539e+18,NaN,NaN,NaN,NaN,NaN
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,NaN,8.355496e+18,NaN,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,NaN,NaN
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,NaN,2.355772e+18,NaN,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,NaN,NaN
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,NaN,6.156971e+18,NaN,NaN,NaN,NaN,NaN


In [5]:
events_df['connection_type'].describe()

count       1809296
unique            4
top       Cable/DSL
freq        1291512
Name: connection_type, dtype: object

In [6]:
events_df['connection_type'].value_counts()

Cable/DSL    1291512
Cellular      517204
Corporate        527
Dialup            53
Name: connection_type, dtype: int64

In [7]:
events_df['connection_type'].isnull().sum()


5935285

In [70]:
auctions_df = auctions_df.sort_values(by=['device_id','date'])
auctions_df['date_dif'] = auctions_df['date'].shift(periods=-1) - auctions_df['date']
auctions_df['in_seconds'] = np.nan
last_row = False
last_index = False
for index, row in auctions_df.iterrows():
    if not(isinstance(last_row, bool)):
        if row['device_id']!=last_row['device_id']:
            auctions_df.at[last_index,'date_dif'] = np.nan
    auctions_df.at[index,'in_seconds'] = row['date_dif'].total_seconds()
    last_row = row
    last_index = index
auctions_df['in_seconds'] = np.where(auctions_df['date_dif'].isnull(), np.nan, auctions_df['in_seconds'])

In [222]:
auctions_df.head(10)

,date,device_id,ref_type_id,source_id
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0
5,2019-04-23 18:58:02.675804,6707090658317158573,1,0
6,2019-04-23 18:58:02.848212,8869722088125970841,1,0
7,2019-04-23 18:58:03.048872,7445213948764639634,1,0
8,2019-04-23 18:58:03.285788,2932617030932207332,1,0
9,2019-04-23 18:58:03.532112,6405811806780450397,7,0


In [75]:
auctions_df.to_csv('data/auctions_seconds.csv')

In [15]:
datos = pd.merge(auctions_df, installs_df, left_on='device_id', right_on='ref_hash', how='left')

In [9]:
installs_df.shape

(481511, 14)

In [10]:
auctions_df.shape

(47409528, 4)

In [17]:
datos.shape

(52602582, 18)

In [18]:
datos.head(5).sort_values(by=['date','device_id'])

,date,device_id,ref_type_id,source_id,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_brand,device_model,user_agent,event_uuid,kind,wifi,trans_id
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0,2019-04-21 17:52:11.181,302,1891515180541284343,2.564673e+18,NaN,False,False,2.987569314309514e+18,4.047342e+18,Dalvik/2.1.0 (Linux; U; Android 7.0; Redmi Not...,NaN,NaN,True,NaN
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0,2019-04-22 00:20:36.406,309,1494519392962156891,4.441122e+18,NaN,False,True,NaN,7.538551e+18,NaN,7eb05e7f-d62d-419f-b732-3d6b6765ee57,app_open,NaN,NaN
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0,2019-04-21 14:14:23.301,210,1891515180541284343,6.416039e+18,NaN,False,False,NaN,5.633254e+18,Dalvik/2.1.0 (Linux; U; Android 7.0; SM-J701M ...,NaN,NaN,NaN,NaN
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [538]:
auctions_1 = auctions_df.loc[auctions_df['date']<'2019-04-20 00:00:00']

In [544]:
# calculate time in seconds
auctions_1.drop_duplicates(inplace=True)
auctions_1 = auctions_1.sort_values(by=['device_id','date'])
auctions_1['date_dif'] = auctions_1['date'].shift(periods=-1) - auctions_1['date']
auctions_1['device_id_next'] = auctions_1['device_id'].astype('object').shift(periods=-1)
auctions_1['date_dif'] = np.where(auctions_1['device_id_next']==auctions_1['device_id'], auctions_1['date_dif'], datetime.datetime(2019,4,20)-auctions_1['date'])
auctions_1['in_seconds'] = auctions_1['date_dif'].dt.total_seconds()
auctions_1['status_censored'] = auctions_1['device_id_next']==auctions_1['device_id']
auctions_1.drop(['device_id_next','date_dif'], axis='columns', inplace=True)
auctions_1['device_id'] = auctions_1['device_id'].astype('object')
#calculate previus time in seconds
auctions_1['date_prev'] = auctions_1['date'].shift()
auctions_1['date_dif_prev'] = auctions_1['date']- auctions_1['date_prev']
auctions_1['device_id_prev'] = auctions_1['device_id'].astype('object').shift()
auctions_1['date_dif_prev'] = np.where(auctions_1['device_id_prev']==auctions_1['device_id'], auctions_1['date_dif_prev'], auctions_1['date']-datetime.datetime(2019,4,18))
auctions_1['last_seen'] = auctions_1['date_dif_prev'].dt.total_seconds()
auctions_1.drop(['device_id_prev','date_dif_prev','date_prev'], axis='columns', inplace=True)
auctions_1 = auctions_1.sort_values(by=['date'])


In [545]:
auctions_1.head(10)

,date,device_id,ref_type_id,source_id,in_seconds,status_censored,last_seen
10129864,2019-04-18 00:00:00.015050,1826643666390887030,7,0,38.041873,True,0.015050
25285906,2019-04-18 00:00:00.029014,7037174172278258682,1,0,394.146288,True,0.029014
3946062,2019-04-18 00:00:00.057540,3392065368947589877,1,1,0.585458,True,0.057540
3946063,2019-04-18 00:00:00.126828,1228982273563226229,1,1,9.739882,True,0.126828
24607726,2019-04-18 00:00:00.132510,4123059034628125459,1,8,21.923076,True,0.132510
11820552,2019-04-18 00:00:00.154022,2544156497510885464,1,1,7.597262,True,0.154022
44538105,2019-04-18 00:00:00.167606,8523620031586497559,1,1,130.324976,True,0.167606
4953159,2019-04-18 00:00:00.176600,3926687473682441392,1,1,0.462632,True,0.176600
11812696,2019-04-18 00:00:00.191287,528082274254399315,1,1,3.438796,True,0.191287
11820553,2019-04-18 00:00:00.217645,8160556786289359332,7,1,60.008722,True,0.217645


In [294]:
installs_1 = installs_df.loc[installs_df['created']<'2019-04-20 00:00:00']
installs_1.head(10)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_brand,user_agent,event_uuid,kind,wifi,trans_id
12,2019-04-18 04:00:27.575,1,1494519392962156891,1084778553542198153,NaN,False,True,NaN,NaN,5d2c31b2-7009-4388-a30c-b8a119b9695f,app_open,NaN,NaN
13,2019-04-18 08:33:46.588,1,1494519392962156891,6412465357609907698,NaN,False,True,NaN,NaN,c4b4577a-e846-4de7-9d40-643d5c66506e,app_open,NaN,NaN
14,2019-04-18 20:38:01.661,1,1494519392962156891,8455113036752892959,NaN,False,True,NaN,NaN,88b28e3e-31bb-4c11-a937-a79ba175cc05,app_open,NaN,NaN
15,2019-04-18 18:51:18.157,1,1494519392962156891,7939167172078327528,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
22,2019-04-19 04:10:34.820,1,1494519392962156891,2251011940536423208,NaN,False,True,NaN,NaN,e9f099f4-47e3-41a6-99e1-e7e6ee154270,app_open,NaN,NaN
23,2019-04-19 04:32:10.018,1,1494519392962156891,8834511447557233212,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
24,2019-04-19 04:31:44.316,1,1494519392962156891,4152412908070003677,NaN,False,True,NaN,NaN,5a53d8bc-edaf-49bd-948a-43cc2b18e40f,app_open,NaN,NaN
25,2019-04-19 04:45:47.500,1,1494519392962156891,5848436212677927803,NaN,False,True,NaN,NaN,54bae8d2-e693-4a0b-9c5f-55fe146ec148,app_open,NaN,NaN
26,2019-04-19 16:57:14.092,1,1494519392962156891,8697087329542169045,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
27,2019-04-19 21:26:47.688,1,1494519392962156891,4137890430319706690,NaN,False,True,NaN,NaN,e6b00fd0-0f10-4b25-a6e4-5053ac135691,app_open,NaN,NaN


In [295]:
installs_df['ref_hash'].describe()

count                  481511
unique                 393565
top       5446085605337844584
freq                       14
Name: ref_hash, dtype: object

In [401]:
installs_1.columns

Index(['created', 'application_id', 'ref_type', 'ref_hash', 'click_hash',
       'attributed', 'implicit', 'device_brand', 'user_agent', 'event_uuid',
       'kind', 'wifi', 'trans_id'],
      dtype='object')

In [546]:
#data X and y
data_full_1 = pd.merge(auctions_1, installs_1, left_on='device_id', right_on='ref_hash', how='inner')
data_full_1 = data_full_1.loc[data_full_1['date']>=data_full_1['created']]
data_full_1['install_diff'] = data_full_1['date']-data_full_1['created']
data_full_1['install_seconds'] = data_full_1['install_diff'].dt.total_seconds()
data_full_1 = data_full_1.loc[data_full_1['in_seconds']>=data_full_1['install_seconds']]
data_x_1 = data_full_1.drop(columns=['in_seconds','status_censored','ref_hash'])
data_y_1 = np.fromiter(zip(data_full_1["status_censored"], data_full_1["in_seconds"]),
                                dtype=[('status_censored', np.bool), ('in_seconds', np.float64)])

In [547]:
class preprocess( BaseEstimator, TransformerMixin ): 
    #Return self nothing else to do here
    def fit( self, X, y = None  ):
        return self
    #Transformer method we wrote for this transformer 
    def transform(self, X , y = None ):
        X = X.copy()
        # boolean transformations
        X['event_uuid'] = np.where(X['event_uuid'].isnull(), 0,1)
        X['click_hash'] = np.where(X['click_hash'].isnull(), 0,1)
        X['Android'] = np.where(X['user_agent'].str.contains('Android', regex=False),1,0)
        X['iOS'] = np.where(X['user_agent'].str.contains('Darwin', regex=False) | X['user_agent'].str.contains('iOS', regex=False),1,0)
        X['trans_id'] = np.where(X['trans_id'].isnull(), 0,1)
        # date transformations
        X['created_weekday'] = X['created'].dt.weekday
        X['created_hour'] = X['created'].dt.hour
        X['created_minute'] = X['created'].dt.minute
        X['date_weekday'] = X['date'].dt.weekday
        X['date_hour'] = X['date'].dt.hour
        X['date_minute'] = X['date'].dt.minute
        X['date_second'] = X['date'].dt.second
        #remove unused columns
        X = X.drop(columns=['date','created', 'install_diff','device_brand','install_seconds','user_agent','device_id'])
        X = pd.get_dummies(X, dummy_na=True, prefix_sep='=')
        self.encoded_columns_ = X.columns
        #returns numpy array
        return X

In [400]:
# format features
data_x_1['event_uuid'] = np.where(data_x_1['event_uuid'].isnull(), 0,1)
data_x_1['click_hash'] = np.where(data_x_1['click_hash'].isnull(), 0,1)
data_x_1['Android'] = np.where(data_x_1['user_agent'].str.contains('Android', regex=False),1,0)
data_x_1['iOS'] = np.where(data_x_1['user_agent'].str.contains('Darwin', regex=False) | data_x_1['user_agent'].str.contains('iOS', regex=False),1,0)
data_x_1['trans_id'] = np.where(data_x_1['trans_id'].isnull(), 0,1)
data_x_1['created_weekday'] = data_x_1['created'].dt.weekday
data_x_1['created_hour'] = data_x_1['created'].dt.hour
data_x_1['created_minute'] = data_x_1['created'].dt.minute
data_x_1['date_weekday'] = data_x_1['date'].dt.weekday
data_x_1['date_hour'] = data_x_1['date'].dt.hour
data_x_1['date_minute'] = data_x_1['date'].dt.minute
data_x_1['date_second'] = data_x_1['date'].dt.second
data_x_1.drop(columns=['date','created', 'install_diff','device_brand','install_seconds','user_agent'], inplace=True)
data_x_1_numeric = pd.get_dummies(data_x_1, dummy_na=True, prefix_sep='=')


In [395]:
data_x_1_numeric.head(5)

,click_hash,attributed,implicit,event_uuid,trans_id,Android,iOS,created_weekday,created_hour,created_minute,...,kind=viewed product listing,kind=viewed shopping card,kind=viewhome,kind=viewitem,kind=viewlisting,kind=wine pages viewed,kind=nan,wifi=false,wifi=true,wifi=nan
7,0,False,True,1,0,1,0,3,4,56,...,0,0,0,0,0,0,0,0,1,0
153,0,False,False,0,0,0,1,4,21,7,...,0,0,0,0,0,0,1,1,0,0
570,0,False,True,1,0,1,0,3,12,53,...,0,0,0,0,0,0,0,0,1,0
571,0,False,False,0,0,1,0,3,12,53,...,0,0,0,0,0,0,1,0,1,0
1134,0,False,False,0,0,1,0,4,17,13,...,0,0,0,0,0,0,1,0,1,0


In [548]:
data_y_1 = np.fromiter(zip(data_full_1.head(100)["status_censored"], data_full_1.head(100)["in_seconds"]),
                                dtype=[('status_censored', np.bool), ('in_seconds', np.float64)])

In [408]:
estimator = CoxPHSurvivalAnalysis(alpha=0.1)
estimator.fit(data_x_1_numeric.head(100), data_y_1)
estimator.score(data_x_1_numeric.head(100), data_y_1)

0.8191370444891571

In [549]:
def fit_and_score_features(X, y):
    n_features = X.shape[1]
    scores = np.empty(n_features)
    m = CoxPHSurvivalAnalysis(alpha=0.1)
    for j in range(n_features):
        Xj = X[:, j:j+1]
        m.fit(Xj, y)
        scores[j] = m.score(Xj, y)
    return scores

scores = fit_and_score_features(data_x_1_numeric.head(100).values, data_y_1)
pd.Series(scores, index=data_x_1_numeric.columns).sort_values(ascending=False)

created_minute                   0.566742
created_hour                     0.538670
kind=nan                         0.535626
implicit                         0.535626
event_uuid                       0.535626
date_weekday                     0.528298
source_id=1                      0.522210
application_id=194               0.518151
application_id=122               0.517700
kind=open                        0.517587
application_id=75                0.517362
application_id=77                0.516009
source_id=0                      0.515671
date_hour                        0.515333
kind=af_app_opened               0.514205
application_id=94                0.512740
created_weekday                  0.512514
source_id=3                      0.512401
application_id=133               0.512289
application_id=68                0.511048
application_id=210               0.509357
source_id=5                      0.509132
application_id=54                0.508906
application_id=214               0

In [559]:
pipe = Pipeline([('preprocess', preprocess()),
                 ('select', SelectKBest(fit_and_score_features, k=50)),
                 ('model', CoxPHSurvivalAnalysis())])

In [560]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

In [562]:
data_x_11 = data_x_1.head(100).copy()
param_grid = {'select__k': np.arange(1, data_x_11.shape[1] + 1), 'model__alpha': 2. ** np.arange(-12, 13, 2)}
gcv = GridSearchCV(pipe, param_grid, return_train_score=True, cv=3, iid=True)
gcv.fit(data_x_11, data_y_1)

pd.DataFrame(gcv.cv_results_).sort_values(by='mean_test_score', ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,param_select__k,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
128,5.890001,0.083724,0.029843,0.000270,1,15,"{'model__alpha': 1.0, 'select__k': 15}",0.551456,0.625571,0.639511,0.604972,0.038826,1,0.819277,0.770330,0.730036,0.773214,0.036490
150,5.933861,0.074766,0.030164,0.000592,4,18,"{'model__alpha': 4.0, 'select__k': 18}",0.541748,0.611872,0.655804,0.602527,0.047133,2,0.816134,0.754559,0.736218,0.768970,0.034180
151,5.906753,0.054564,0.029925,0.000306,4,19,"{'model__alpha': 4.0, 'select__k': 19}",0.543689,0.614155,0.649695,0.601925,0.044221,3,0.817182,0.755052,0.735188,0.769141,0.034925
149,5.903093,0.080928,0.029735,0.000176,4,17,"{'model__alpha': 4.0, 'select__k': 17}",0.545631,0.605023,0.651731,0.600243,0.043548,4,0.815086,0.756530,0.726945,0.766187,0.036626
148,5.915458,0.053529,0.030033,0.000709,4,16,"{'model__alpha': 4.0, 'select__k': 16}",0.543689,0.605023,0.649695,0.598911,0.043591,5,0.804610,0.757023,0.725914,0.762516,0.032361
109,5.913734,0.088543,0.029849,0.000239,0.25,15,"{'model__alpha': 0.25, 'select__k': 15}",0.530097,0.636986,0.631365,0.598789,0.049356,6,0.822944,0.770330,0.734158,0.775811,0.036453
146,5.912234,0.079419,0.029767,0.000232,4,14,"{'model__alpha': 4.0, 'select__k': 14}",0.545631,0.600457,0.645621,0.596720,0.041002,7,0.795705,0.758995,0.719732,0.758144,0.031021
127,5.892765,0.058398,0.029623,0.000066,1,14,"{'model__alpha': 1.0, 'select__k': 14}",0.537864,0.625571,0.627291,0.596318,0.041961,8,0.813515,0.770330,0.721278,0.768374,0.037681
147,5.920654,0.076365,0.030150,0.000428,4,15,"{'model__alpha': 4.0, 'select__k': 15}",0.545631,0.600457,0.643585,0.596048,0.040204,9,0.797800,0.758502,0.728490,0.761597,0.028380
110,5.920170,0.066333,0.030100,0.000085,0.25,16,"{'model__alpha': 0.25, 'select__k': 16}",0.512621,0.636986,0.637475,0.594863,0.059029,10,0.838659,0.770330,0.735188,0.781392,0.042960


In [563]:
gcv.best_params_

{'model__alpha': 1.0, 'select__k': 15}

In [564]:
data_x_11 = data_x_1.head(100).copy()
pipe.set_params(**gcv.best_params_)
pipe.fit(data_x_11, data_y_1)

encoder, transformer, final_estimator = [s[1] for s in pipe.steps]
pd.Series(final_estimator.coef_, index=encoder.encoded_columns_[transformer.get_support()])

last_seen                      -0.000039
created_weekday                 0.760496
date_hour                       0.016006
date_second                     0.021819
ref_type_id=1                   0.117072
ref_type_id=7                  -0.117072
source_id=1                    -0.890550
source_id=3                     0.539926
source_id=7                    -1.156081
application_id=121             -0.333614
application_id=210             -0.006043
ref_type=1494519392962156891   -0.117072
ref_type=1891515180541284343    0.117072
kind=open                      -0.107620
wifi=nan                        0.273920
dtype: float64

In [565]:
data_x_11 = data_x_1.head(100).copy()
pipe.score(data_x_11, data_y_1)

0.7289740698985344

In [490]:
data_x_11.columns

Index(['date', 'ref_type_id', 'source_id', 'created', 'application_id',
       'ref_type', 'click_hash', 'attributed', 'implicit', 'device_brand',
       'user_agent', 'event_uuid', 'kind', 'wifi', 'trans_id', 'install_diff',
       'install_seconds'],
      dtype='object')

In [485]:
data_y_1.shape

(100,)

In [333]:
data_x_1_numeric.shape

(31377, 1601)

In [336]:
data_y_1.shape

(31377,)

In [ ]:
test1 = auctions_1[['device_id','date']]

In [89]:
test1 = test1.sort_values(by=['device_id','date'])

In [90]:
test1 = test1.head(1000)

In [91]:
test1.shape

(1000, 2)

In [92]:
test1['date_dif'] = test1['date'].shift(periods=-1) - test1['date']

In [97]:
test1['device_id_next'] = test1['device_id'].astype('object').shift(periods=-1)

In [99]:
test1['date_dif'] = np.where(test1['device_id_next']==test1['device_id'], test1['date_dif'], datetime.datetime(2019,4,20)-test1['date'])

In [101]:
test1['in_seconds'] = test1['date_dif'].dt.total_seconds()

In [103]:
test1['status-censored'] = test1['device_id_next']==test1['device_id']

In [104]:
test1.head(50)

,device_id,date,date_dif,in_seconds,device_id_next,status-censored
30900602,41863526108385,2019-04-19 19:40:28.465866,0 days 04:19:31.534134,15571.534134,161514654074162,False
46872336,161514654074162,2019-04-18 02:52:46.357746,0 days 00:00:28.744020,28.744020,161514654074162,True
7330779,161514654074162,2019-04-18 02:53:15.101766,0 days 00:00:25.982516,25.982516,161514654074162,True
16177030,161514654074162,2019-04-18 02:53:41.084282,0 days 00:00:18.239779,18.239779,161514654074162,True
19683863,161514654074162,2019-04-18 02:53:59.324061,0 days 00:00:02.851261,2.851261,161514654074162,True
22213367,161514654074162,2019-04-18 02:54:02.175322,0 days 00:00:47.365665,47.365665,161514654074162,True
20455826,161514654074162,2019-04-18 02:54:49.540987,1 days 21:05:10.459013,162310.459013,186034136943920,False
16127540,186034136943920,2019-04-18 16:42:46.331894,0 days 02:40:55.874138,9655.874138,186034136943920,True
4859451,186034136943920,2019-04-18 19:23:42.206032,1 days 04:36:17.793968,102977.793968,283297668933729,False
293739,283297668933729,2019-04-18 22:24:44.520583,1 days 01:35:15.479417,92115.479417,345999128501141,False


In [61]:
test1['date_dif'] = test1['date'].shift(periods=-1) - test1['date']

In [69]:
test1.head(50)

,date,device_id,date_dif,in_seconds
36601349,2019-04-23 15:00:57.222979,4.062141e+13,NaT,NaN
30900602,2019-04-19 19:40:28.465866,4.186353e+13,07:11:58.427014,25918.427014
14875535,2019-04-20 02:52:26.892880,4.186353e+13,00:06:35.616350,395.616350
32062481,2019-04-20 02:59:02.509230,4.186353e+13,00:06:59.166558,419.166558
45613542,2019-04-20 03:06:01.675788,4.186353e+13,00:02:55.712372,175.712372
42215232,2019-04-20 03:08:57.388160,4.186353e+13,00:02:29.075743,149.075743
22374394,2019-04-20 03:11:26.463903,4.186353e+13,00:00:14.547768,14.547768
12690007,2019-04-20 03:11:41.011671,4.186353e+13,00:00:45.669391,45.669391
42215538,2019-04-20 03:12:26.681062,4.186353e+13,00:00:30.801420,30.801420
12690107,2019-04-20 03:12:57.482482,4.186353e+13,00:01:58.661626,118.661626
